Homework 1

Data source: the NYC taxi dataset

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
pd.__version__, np.__version__, sns.__version__

('2.2.3', '2.2.5', '0.13.2')

In [6]:
import mlflow
mlflow.__version__

'2.22.0'

In [2]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('NYC-taxi-experiment')

2025/05/08 11:03:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/08 11:03:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/Users/nabe/Desktop/Research/Formations/MLOps/DataTalks_MLOPS/mlops-zoomcamp/02_experiment_tracking/mlruns/1', creation_time=1746695003354, experiment_id='1', last_update_time=1746695003354, lifecycle_stage='active', name='NYC-taxi-experiment', tags={}>

In [7]:
jan_fn =  '../data/yellow_tripdata_2023-01.parquet'
feb_fn =  '../data/yellow_tripdata_2023-02.parquet'

In [8]:
january_trips = pd.read_parquet(
   jan_fn
)
january_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [9]:
january_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [10]:
# Q1: how many columns are there
print(f"# samples (rows) = {january_trips.shape[0]} and # features (columns) = {january_trips.shape[1]}")

# samples (rows) = 3066766 and # features (columns) = 19


In [11]:
# Q2: computing duration in minutes
# from the info, we see that the pickup and dropoff columns are identified as datetieme
january_trips['duration'] = (
    january_trips['tpep_dropoff_datetime'] - january_trips['tpep_pickup_datetime']
).dt.total_seconds().div(60.)


In [12]:
# what is the standard deviation of the trips duration in january
print(f"Standard deviation : {january_trips.duration.std()}")
print(f"Average : {january_trips.duration.mean()}")
print(f"Mode : {january_trips.duration.mode().iloc[0]}")
print(f"Min = {january_trips.duration.min()} --- Max = {january_trips.duration.max()}")

Standard deviation : 42.59435124195458
Average : 15.668995167330452
Mode : 7.0
Min = -29.2 --- Max = 10029.183333333332


We can see that there are some outliers, negative and too long trip duratioons.

These skew the data distribution leading to a big variation.

In [13]:
# Q3: drop outliers (any trip not between 1 and 60 minutes inclusive)
filtered_january_trips = january_trips[
    (1 <= january_trips.duration) & (january_trips.duration <= 60.)
]
print(f"Fraction of records left after filtering : {filtered_january_trips.shape[0] / january_trips.shape[0]:.2f}")

Fraction of records left after filtering : 0.98


"Only 2%" was bad data, which is not that bad.

In [14]:
#Q4: one-hot encoding
categorical_cols = ['PULocationID', 'DOLocationID']
display(filtered_january_trips[categorical_cols].head())
# check for NAs
print(f"# NA :\n{filtered_january_trips[categorical_cols].isna().sum()}")

,PULocationID,DOLocationID
0,161,141
1,43,237
2,48,238
3,138,7
4,107,79


# NA :
PULocationID    0
DOLocationID    0
dtype: int64


In [15]:
training_feats_dict = (
    filtered_january_trips[['PULocationID', 'DOLocationID']]
    .astype(str).to_dict('records')
)
training_feats_dict[:2]

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'}]

In [16]:
from sklearn.feature_extraction import DictVectorizer

In [17]:
vectorizer = DictVectorizer(sparse=False)
vectorizer.fit(training_feats_dict)

DictVectorizer(sparse=False)

In [18]:
X = vectorizer.transform(training_feats_dict)
X[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(2, 515))

In [19]:
print(f"# features = {X.shape[1]}")

# features = 515


In [20]:
# Q5: Training a model
# For computation issue, choose a subset of data randomly
# so that I can run it on my laptop
# for reproductibility, fix the random seed
np.random.seed(42)
N = 100_000
train_indices = np.random.choice(range(X.shape[0]), N, replace=False)
train_indices[:5]

array([1475569,  183006, 2788577,  394364, 1123812])

In [21]:
# target is duration
X = X[train_indices]
y = filtered_january_trips['duration'].to_numpy()[train_indices]

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import root_mean_squared_error

In [25]:
lr_model = LinearRegression()
lr_model.fit(X, y)

LinearRegression()

In [26]:
rmse = root_mean_squared_error(
    y, lr_model.predict(X)
)
print(f'RMSE on train = {rmse:.2f}')

RMSE on train = 7.61


In [27]:
# save model
import pickle
with open('../models/linear_reg.pkl', 'wb') as fw:
    pickle.dump((vectorizer, lr_model), fw, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
# embed all the preprocessing in a function
def preprocess_df(fn, categorical_cols):
    df = pd.read_parquet(fn)
    # create duration (in minutes) feature
    df['duration'] = (
        df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    ).dt.total_seconds().div(60.)
    # filter out outliers: trips should be between 1 and 60 minutes.
    df = df[
        (1 <= df.duration) & (df.duration <= 60.)
    ]
    # convert categorical columns to string data type
    df[categorical_cols] = df[categorical_cols].astype(str)
    return df

In [30]:
# Q5: evaluating the model
# validation data: february 2023
# here again, I keep N rows
february_trips = preprocess_df(feb_fn, categorical_cols)
february_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [31]:
print(f"Standard deviation of trips duration in February : {february_trips.duration.std()}")
print(f"Average of trips duration in February : {february_trips.duration.mean()}")
print(f"Mode of trips duration in February : {february_trips.duration.mode().iloc[0]}")
print(f"Min = {february_trips.duration.min()} --- Max = {february_trips.duration.max()}")

Standard deviation of trips duration in February : 10.064227216640147
Average of trips duration in February : 14.468110313050428
Mode of trips duration in February : 7.0
Min = 1.0 --- Max = 60.0


In [32]:
validation_features_dict = february_trips[categorical_cols].to_dict('records')

In [33]:
X_valid = vectorizer.transform(validation_features_dict)

In [34]:
# check dimension
print(f'Validation set dimension: {X_valid.shape}')
print(f'Does it match training feature vector dimension ? {X.shape[1] == X_valid.shape[1]}')

Validation set dimension: (2855951, 515)
Does it match training feature vector dimension ? True


In [35]:
valid_indices = np.random.choice(range(X_valid.shape[0]), N, replace=False)
X_valid = X_valid[valid_indices]
y_valid = february_trips.duration.to_numpy()[valid_indices]

In [36]:
valid_rmse = root_mean_squared_error(
    y_valid, lr_model.predict(X_valid)
)
print(f'Validation set RMSE: {valid_rmse:.2f}')

Validation set RMSE: 7.86


In [37]:
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X, y)
root_mean_squared_error(y_valid, lasso_model.predict(X_valid))

8.786466576483576

Important to note that at this stage there is no tracking. So changing anything in the model would result in not having the history of what we changed. That's where mlflow comes to play.

In [38]:
with mlflow.start_run():
    mlflow.set_tag("developer", "mamady")
    mlflow.log_param('train-data-path', '../data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param('valid-data-path', '../data/yellow_tripdata_2023-02.parquet')

    alpha = 0.01
    mlflow.log_param('alpha', alpha)
    lasso_model = Lasso(alpha=0.1)
    lasso_model.fit(X, y)
    
    rmse = root_mean_squared_error(
        y_valid, lasso_model.predict(X)
    )
    mlflow.log_metric('rmse', rmse)